### Версия питона 3.8.10
#### Добавьте все необходимые библиотеки сами

In [ ]:
# !pip install catboost=="1.2.8"
# !pip install pandas=="2.0.3"
# !pip install numpy=="1.24.3"
# !pip install scikit-learn=="2.3.0"
# !pip install scipy=="1.10.0"


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

df_s = pd.read_csv("sample_submission.csv")
df_test = pd.read_csv("test.csv")
d_train = pd.read_csv("train.csv")
d_train.dropna(inplace=True)

#### добавляем флаг спец подстроки у ссылки сайта

In [3]:
def func_flag(x):
    if "porn" in x or "sex" in x:
        return 1
    else:
        return 0
d_train["flg"] = d_train["url"].apply(func=func_flag)
df_test["flg"] = df_test["url"].apply(func=func_flag)

In [ ]:
X_train, y_train = d_train.drop(columns=["label"]), d_train['label']
X_test = df_test


#### tfidf(лог шкала у частоты tf + нграммы от 1 до 3 + l2 нормировка)+truncatedSVD для уменшьения размерности

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3), norm='l2')
sparse_data = vectorizer.fit_transform(X_train['title'].to_list())

svd = TruncatedSVD(n_components=400, random_state=42)
X_reduced = svd.fit_transform(sparse_data)

X_reduced = normalize(X_reduced, norm='l2', axis=1)

X_reduced = np.hstack((X_reduced, X_train["flg"].values.reshape(-1, 1)))

X_reduced = csr_matrix(X_reduced)

In [ ]:
sparse_data = vectorizer.transform(X_test['title'].to_list())
X_reduced_test = svd.transform(sparse_data)

X_reduced_test = normalize(X_reduced_test, norm='l2', axis=1)

X_reduced_test = np.hstack((X_reduced_test, X_test["flg"].values.reshape(-1, 1)))

X_reduced_test = csr_matrix(X_reduced_test)

In [ ]:
from catboost import CatBoostClassifier
from catboost import Pool



train_pool = Pool(X_reduced, y_train)
model = CatBoostClassifier(iterations=700, verbose=50, learning_rate=0.1, l2_leaf_reg=1, max_depth=6)
model.fit(train_pool)

0:	learn: 0.5308837	total: 90.7ms	remaining: 1m 3s
50:	learn: 0.0630463	total: 1.6s	remaining: 20.3s
100:	learn: 0.0501972	total: 3.08s	remaining: 18.3s
150:	learn: 0.0430631	total: 4.6s	remaining: 16.7s
200:	learn: 0.0383181	total: 6.1s	remaining: 15.2s
250:	learn: 0.0346648	total: 7.57s	remaining: 13.5s
300:	learn: 0.0318297	total: 9.01s	remaining: 11.9s
350:	learn: 0.0295206	total: 10.5s	remaining: 10.4s
400:	learn: 0.0275263	total: 11.9s	remaining: 8.86s
450:	learn: 0.0256799	total: 13.4s	remaining: 7.38s
500:	learn: 0.0240333	total: 14.8s	remaining: 5.88s
550:	learn: 0.0226604	total: 16.2s	remaining: 4.39s
600:	learn: 0.0213517	total: 17.6s	remaining: 2.91s
650:	learn: 0.0201291	total: 19.1s	remaining: 1.44s
699:	learn: 0.0190040	total: 20.5s	remaining: 0us


In [13]:
df_s["label"] = model.predict(X_reduced_test)

In [19]:
df_s.set_index("ID").to_csv("submit_v1.csv")